In [1]:
# Implementation for the dataset for the competition https://www.kaggle.com/c/dogs-vs-cats
# On image cat-dog classification
# Inspared on: https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d

import  numpy  as np
import  matplotlib.pyplot  as plt
import h5py
import scipy
import os, os.path
import keras
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [3]:
img_size = 150
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_size, img_size)
else:
    input_shape = (img_size, img_size, 3)

#To generate the train_datagen from flow_from_directory, the dataset must be in a directory with two subdirectories.
#Example: Train path: dataset/all/train
#         Train subdirectories:  1) dataset/all/train/cat  2) dataset/all/train/dog

train_dir = 'dataset/all/train'
validation_dir = 'dataset/all/validation/'
DIR = 'dataset/all/train'

numImages = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

In [4]:
#Example using dataFlow from Image
train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')


Found 3000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [5]:
n_train = 4000
n_validation = 2000
epochs = 20

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=n_train // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=n_validation // batch_size)

Epoch 1/20
125/125 [==============================] - 295s 2s/step - loss: 0.6944 - acc: 0.6567 - val_loss: 0.6998 - val_acc: 0.6667
Epoch 2/20
125/125 [==============================] - 247s 2s/step - loss: 0.6269 - acc: 0.6655 - val_loss: 0.5987 - val_acc: 0.6893
Epoch 3/20
125/125 [==============================] - 242s 2s/step - loss: 0.5840 - acc: 0.7003 - val_loss: 0.5581 - val_acc: 0.7073
Epoch 4/20
125/125 [==============================] - 243s 2s/step - loss: 0.5755 - acc: 0.7146 - val_loss: 0.5422 - val_acc: 0.7330
Epoch 5/20
125/125 [==============================] - 241s 2s/step - loss: 0.5417 - acc: 0.7377 - val_loss: 0.5476 - val_acc: 0.7453
Epoch 6/20
125/125 [==============================] - 241s 2s/step - loss: 0.5351 - acc: 0.7385 - val_loss: 0.5146 - val_acc: 0.7570
Epoch 7/20
125/125 [==============================] - 242s 2s/step - loss: 0.5171 - acc: 0.7505 - val_loss: 0.4994 - val_acc: 0.7647
Epoch 8/20
125/125 [==============================] - 1589s 13s/step 

In [6]:
model.save_weights('catDog_CNN.h5')

In [16]:
# list all data in history
print(model.history)